In [9]:
import numpy as np
import os
import csv
import runtests
import makegraphs

In [2]:
def return_one(startdisk, zfsname, csvfilename, ip, skip_num=0):
    with open(csvfilename) as csvfile:
        reader = csv.reader(csvfile, delimiter=",")
        next(reader)
        for _ in range(skip_num):
            next(reader)
        row = next(reader)
        testname, numberdisks, raidmode, raid0, ashift, compression, recordsize, atime, filesize, benchmark, runtime, blocksizes, iodepths, numjobs = row
        print (testname, numberdisks, raidmode, raid0, ashift, compression, recordsize, atime, filesize, benchmark, runtime, blocksizes, iodepths, numjobs)
        print ("++++++++++++++++++++++++++++")
        print (testname)
        print ("++++++++++++++++++++++++++++")
        numberdisks = int(numberdisks)
        raidmode = int(raidmode)
        raid0 = int(raid0)
        return runtests.make_commands(startdisk, numberdisks, zfsname, raidmode, raid0, ashift, compression, recordsize, atime, testname, ip, filesize, benchmark, runtime, blocksizes, iodepths, numjobs)

In [3]:
cmds = return_one("c", "tank", "config.csv", "192.168.169.207", skip_num=11)

twodisk 2 0 0 12  128k off 100g True 100 32m 64m 1 4 1 4
++++++++++++++++++++++++++++
twodisk
++++++++++++++++++++++++++++


In [4]:
for cmd in cmds:
    print(cmd)

mkdir twodisk
echo -------------------------------
echo make zfs
echo -------------------------------
zpool create tank -o ashift=12  sdc sdd
zfs set recordsize=128k atime=off tank
echo -------------------------------
echo test zfs
echo -------------------------------
./bench_fio  --type directory --quiet -m write read --loops 1 --target /tank -o twodisk_zfs -b 32m 64m --iodepth 1 4 --numjobs 1 4 --size 100g --runtime 100 --ioengine psync --loginterval 1250
echo -------------------------------
echo make lustre
echo -------------------------------
mkfs.lustre --backfstype=zfs --fsname=test --index=0 --reformat --mgs --mdt mgsmdt/mgsmdt
mkfs.lustre --backfstype=zfs --fsname=test --index=0 --ost --mgsnode=192.168.169.207@tcp tank/ost0
mount -t lustre mgsmdt/mgsmdt /mnt/mgsmdt
mount -t lustre tank/ost0 /mnt/ost
mount -t lustre 192.168.169.207@tcp:/test /mnt/lustre
echo -------------------------------
echo test lustre
echo -------------------------------
./bench_fio  --type directory --quie

# make sure the change device, name, rw
"fio --directory=" "--direct=1 --rw=write --bs=32m --ioengine=libaio --iodepth=64 --filesize=100g --runtime=120 --time_based --group_reporting --name=throughput-test --eta-newline=1"
"fio --directory=" "--direct=1 --rw=read --bs=32m --ioengine=libaio --iodepth=64 --filesize=100g --runtime=120 --time_based --group_reporting --name=throughput-test --eta-newline=1 --readonly"

fio --directory=/tank --direct=1 --rw=write --bs=32m --ioengine=libaio --iodepth=64 --filesize=30g --numjobs=2 --group_reporting --name=throughput-test --eta-newline=1

./bench_fio --target /tank --type directory -o onedisk_fioplot -b 16m 32m 64m -m write --loops 1 --iodepth 1 2 --numjobs 1 2 --ss iops:10% --ss-ramp 10 --ss-dur 30 --runtime 300 --size 20g

./fio_plot/fio_plot -T "title" -r write -g -t bw -d 1 2 -n 1 2 -i benchmark_script/onedisk_fioplot/tank/32m/

./fio-plot/fio_plot/fio_plot -T "bandwidth of write on lustre raidz2(8 drives) given iodepth and numjobs 32m" -i z2_one_8_single_lustre/lustre/16m -g -r write -t bw -d 1 2 4 -n 1 4 8
./fio-plot/fio_plot/fio_plot -T "bandwidth of write on zfs 60 (2x8 drives) given iodepth and numjobs 64m" -i z2_one_8_single_zfs/tank/64m -g -r write -t bw -d 1 2 4 -n 1 4 8
./fio-plot/fio_plot/fio_plot -T "bandwidth of read on zfs raidz2(8 drives) given iodepth and numjobs m" -i z2_one_8_single_zfs/tank/16m -g -r read -t bw -d 1 2 4 -n 1 4 8


In [10]:
skip_num = 11
# TODO: stop to param
# set stop to 0 if you don't need it
stop = 0
cmds = makegraphs.make_commands(skip_num, stop)
for cmd in cmds:
    print (cmd)

mkdir graph_twodisk
mkdir graph_twodisk/32m
./fio-plot/fio_plot/fio_plot -T "bandwidth of read on zfs twodisk 32m" -i twodisk_lustre/lustre/32m -g -r read -t bw -d 1 4 -n 1 4 --disable-fio-version
mv bandwidth-of-read-on-zfs-twodisk-32m* graph_twodisk/32m
./fio-plot/fio_plot/fio_plot -T "bandwidth of write on zfs twodisk 32m" -i twodisk_lustre/lustre/32m -g -r write -t bw -d 1 4 -n 1 4 --disable-fio-version
mv bandwidth-of-write-on-zfs-twodisk-32m* graph_twodisk/32m
./fio-plot/fio_plot/fio_plot -T "bandwidth of read on lustre twodisk 32m" -i twodisk_zfs/tank/32m -g -r read -t bw -d 1 4 -n 1 4 --disable-fio-version
mv bandwidth-of-read-on-lustre-twodisk-32m* graph_twodisk/32m
./fio-plot/fio_plot/fio_plot -T "bandwidth of write on lustre twodisk 32m" -i twodisk_zfs/tank/32m -g -r write -t bw -d 1 4 -n 1 4 --disable-fio-version
mv bandwidth-of-write-on-lustre-twodisk-32m* graph_twodisk/32m
mkdir graph_twodisk/64m
./fio-plot/fio_plot/fio_plot -T "bandwidth of read on zfs twodisk 64m" -i tw